In [1]:
import argparse
import os
import shutil
import time

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import Dataset

import densenet as dn

# Imports
from torch.autograd import Variable
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

class SRCIFAR100(Dataset):

    def __init__(self, folder_path, transform):
        """
        Args:
            csv_file (string): Path to the image file with included label file.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.folder_path = folder_path
        self.labels = np.load(self.folder_path + '/labels.npy')
        self.transform = transform

    def __len__(self):
        return self.labels.shape[0]

    def __getitem__(self, idx):
        img_name = os.path.join(self.folder_path,str(idx)+'.npy')
        img_arr = np.uint8(np.load(img_name)*255)
        img_arr = np.rollaxis(img_arr,0,3)
        image = Image.fromarray(img_arr)

        if self.transform:
            image = self.transform(image)
            
        target = torch.LongTensor(np.array(self.labels[idx]))

        return image, target


# used for logging to TensorBoard
# from tensorboard_logger import configure, log_value

parser = argparse.ArgumentParser(description='PyTorch DenseNet Training')
parser.add_argument('--epochs', default=300, type=int,
                    help='number of total epochs to run')
parser.add_argument('--start-epoch', default=0, type=int,
                    help='manual epoch number (useful on restarts)')
parser.add_argument('-b', '--batch-size', default=64, type=int,
                    help='mini-batch size (default: 64)')
parser.add_argument('--lr', '--learning-rate', default=0.1, type=float,
                    help='initial learning rate')
parser.add_argument('--momentum', default=0.9, type=float, help='momentum')
parser.add_argument('--weight-decay', '--wd', default=1e-4, type=float,
                    help='weight decay (default: 1e-4)')
parser.add_argument('--print-freq', '-p', default=10, type=int,
                    help='print frequency (default: 10)')
parser.add_argument('--layers', default=100, type=int,
                    help='total number of layers (default: 100)')
parser.add_argument('--growth', default=12, type=int,
                    help='number of new channels per layer (default: 12)')
parser.add_argument('--droprate', default=0, type=float,
                    help='dropout probability (default: 0.0)')
parser.add_argument('--no-augment', dest='augment', action='store_false',
                    help='whether to use standard augmentation (default: True)')
parser.add_argument('--reduce', default=0.5, type=float,
                    help='compression rate in transition stage (default: 0.5)')
parser.add_argument('--no-bottleneck', dest='bottleneck', action='store_false',
                    help='To not use bottleneck block')
parser.add_argument('--resume', default='', type=str,
                    help='path to latest checkpoint (default: none)')
parser.add_argument('--name', default='DenseNet_BC_100_12', type=str,
                    help='name of experiment')
# parser.add_argument('--tensorboard',
#                     help='Log progress to TensorBoard', action='store_true')
parser.set_defaults(bottleneck=True)
parser.set_defaults(augment=True)

best_prec1 = 0

dbpn_args = ['feat0.conv.weight', 'feat0.conv.bias', 'feat0.act.weight', 'feat1.conv.weight', 'feat1.conv.bias', 'feat1.act.weight', 'up1.up_conv1.deconv.weight', 'up1.up_conv1.deconv.bias', 'up1.up_conv1.act.weight', 'up1.up_conv2.conv.weight', 'up1.up_conv2.conv.bias', 'up1.up_conv2.act.weight', 'up1.up_conv3.deconv.weight', 'up1.up_conv3.deconv.bias', 'up1.up_conv3.act.weight', 'down1.down_conv1.conv.weight', 'down1.down_conv1.conv.bias', 'down1.down_conv1.act.weight', 'down1.down_conv2.deconv.weight', 'down1.down_conv2.deconv.bias', 'down1.down_conv2.act.weight', 'down1.down_conv3.conv.weight', 'down1.down_conv3.conv.bias', 'down1.down_conv3.act.weight', 'up2.up_conv1.deconv.weight', 'up2.up_conv1.deconv.bias', 'up2.up_conv1.act.weight', 'up2.up_conv2.conv.weight', 'up2.up_conv2.conv.bias', 'up2.up_conv2.act.weight', 'up2.up_conv3.deconv.weight', 'up2.up_conv3.deconv.bias', 'up2.up_conv3.act.weight', 'down2.conv.conv.weight', 'down2.conv.conv.bias', 'down2.conv.act.weight', 'down2.down_conv1.conv.weight', 'down2.down_conv1.conv.bias', 'down2.down_conv1.act.weight', 'down2.down_conv2.deconv.weight', 'down2.down_conv2.deconv.bias', 'down2.down_conv2.act.weight', 'down2.down_conv3.conv.weight', 'down2.down_conv3.conv.bias', 'down2.down_conv3.act.weight', 'up3.conv.conv.weight', 'up3.conv.conv.bias', 'up3.conv.act.weight', 'up3.up_conv1.deconv.weight', 'up3.up_conv1.deconv.bias', 'up3.up_conv1.act.weight', 'up3.up_conv2.conv.weight', 'up3.up_conv2.conv.bias', 'up3.up_conv2.act.weight', 'up3.up_conv3.deconv.weight', 'up3.up_conv3.deconv.bias', 'up3.up_conv3.act.weight', 'down3.conv.conv.weight', 'down3.conv.conv.bias', 'down3.conv.act.weight', 'down3.down_conv1.conv.weight', 'down3.down_conv1.conv.bias', 'down3.down_conv1.act.weight', 'down3.down_conv2.deconv.weight', 'down3.down_conv2.deconv.bias', 'down3.down_conv2.act.weight', 'down3.down_conv3.conv.weight', 'down3.down_conv3.conv.bias', 'down3.down_conv3.act.weight', 'up4.conv.conv.weight', 'up4.conv.conv.bias', 'up4.conv.act.weight', 'up4.up_conv1.deconv.weight', 'up4.up_conv1.deconv.bias', 'up4.up_conv1.act.weight', 'up4.up_conv2.conv.weight', 'up4.up_conv2.conv.bias', 'up4.up_conv2.act.weight', 'up4.up_conv3.deconv.weight', 'up4.up_conv3.deconv.bias', 'up4.up_conv3.act.weight', 'down4.conv.conv.weight', 'down4.conv.conv.bias', 'down4.conv.act.weight', 'down4.down_conv1.conv.weight', 'down4.down_conv1.conv.bias', 'down4.down_conv1.act.weight', 'down4.down_conv2.deconv.weight', 'down4.down_conv2.deconv.bias', 'down4.down_conv2.act.weight', 'down4.down_conv3.conv.weight', 'down4.down_conv3.conv.bias', 'down4.down_conv3.act.weight', 'up5.conv.conv.weight', 'up5.conv.conv.bias', 'up5.conv.act.weight', 'up5.up_conv1.deconv.weight', 'up5.up_conv1.deconv.bias', 'up5.up_conv1.act.weight', 'up5.up_conv2.conv.weight', 'up5.up_conv2.conv.bias', 'up5.up_conv2.act.weight', 'up5.up_conv3.deconv.weight', 'up5.up_conv3.deconv.bias', 'up5.up_conv3.act.weight', 'down5.conv.conv.weight', 'down5.conv.conv.bias', 'down5.conv.act.weight', 'down5.down_conv1.conv.weight', 'down5.down_conv1.conv.bias', 'down5.down_conv1.act.weight', 'down5.down_conv2.deconv.weight', 'down5.down_conv2.deconv.bias', 'down5.down_conv2.act.weight', 'down5.down_conv3.conv.weight', 'down5.down_conv3.conv.bias', 'down5.down_conv3.act.weight', 'up6.conv.conv.weight', 'up6.conv.conv.bias', 'up6.conv.act.weight', 'up6.up_conv1.deconv.weight', 'up6.up_conv1.deconv.bias', 'up6.up_conv1.act.weight', 'up6.up_conv2.conv.weight', 'up6.up_conv2.conv.bias', 'up6.up_conv2.act.weight', 'up6.up_conv3.deconv.weight', 'up6.up_conv3.deconv.bias', 'up6.up_conv3.act.weight', 'down6.conv.conv.weight', 'down6.conv.conv.bias', 'down6.conv.act.weight', 'down6.down_conv1.conv.weight', 'down6.down_conv1.conv.bias', 'down6.down_conv1.act.weight', 'down6.down_conv2.deconv.weight', 'down6.down_conv2.deconv.bias', 'down6.down_conv2.act.weight', 'down6.down_conv3.conv.weight', 'down6.down_conv3.conv.bias', 'down6.down_conv3.act.weight', 'up7.conv.conv.weight', 'up7.conv.conv.bias', 'up7.conv.act.weight', 'up7.up_conv1.deconv.weight', 'up7.up_conv1.deconv.bias', 'up7.up_conv1.act.weight', 'up7.up_conv2.conv.weight', 'up7.up_conv2.conv.bias', 'up7.up_conv2.act.weight', 'up7.up_conv3.deconv.weight', 'up7.up_conv3.deconv.bias', 'up7.up_conv3.act.weight', 'output_conv.conv.weight', 'output_conv.conv.bias', 'conv1.weight']
densenet_args = ['block1.layer.0.bn1.weight', 'block1.layer.0.bn1.bias', 'block1.layer.0.bn1.running_mean', 'block1.layer.0.bn1.running_var', 'block1.layer.0.conv1.weight', 'block1.layer.1.bn1.weight', 'block1.layer.1.bn1.bias', 'block1.layer.1.bn1.running_mean', 'block1.layer.1.bn1.running_var', 'block1.layer.1.conv1.weight', 'block1.layer.2.bn1.weight', 'block1.layer.2.bn1.bias', 'block1.layer.2.bn1.running_mean', 'block1.layer.2.bn1.running_var', 'block1.layer.2.conv1.weight', 'block1.layer.3.bn1.weight', 'block1.layer.3.bn1.bias', 'block1.layer.3.bn1.running_mean', 'block1.layer.3.bn1.running_var', 'block1.layer.3.conv1.weight', 'block1.layer.4.bn1.weight', 'block1.layer.4.bn1.bias', 'block1.layer.4.bn1.running_mean', 'block1.layer.4.bn1.running_var', 'block1.layer.4.conv1.weight', 'block1.layer.5.bn1.weight', 'block1.layer.5.bn1.bias', 'block1.layer.5.bn1.running_mean', 'block1.layer.5.bn1.running_var', 'block1.layer.5.conv1.weight', 'block1.layer.6.bn1.weight', 'block1.layer.6.bn1.bias', 'block1.layer.6.bn1.running_mean', 'block1.layer.6.bn1.running_var', 'block1.layer.6.conv1.weight', 'block1.layer.7.bn1.weight', 'block1.layer.7.bn1.bias', 'block1.layer.7.bn1.running_mean', 'block1.layer.7.bn1.running_var', 'block1.layer.7.conv1.weight', 'block1.layer.8.bn1.weight', 'block1.layer.8.bn1.bias', 'block1.layer.8.bn1.running_mean', 'block1.layer.8.bn1.running_var', 'block1.layer.8.conv1.weight', 'block1.layer.9.bn1.weight', 'block1.layer.9.bn1.bias', 'block1.layer.9.bn1.running_mean', 'block1.layer.9.bn1.running_var', 'block1.layer.9.conv1.weight', 'block1.layer.10.bn1.weight', 'block1.layer.10.bn1.bias', 'block1.layer.10.bn1.running_mean', 'block1.layer.10.bn1.running_var', 'block1.layer.10.conv1.weight', 'block1.layer.11.bn1.weight', 'block1.layer.11.bn1.bias', 'block1.layer.11.bn1.running_mean', 'block1.layer.11.bn1.running_var', 'block1.layer.11.conv1.weight', 'trans1.bn1.weight', 'trans1.bn1.bias', 'trans1.bn1.running_mean', 'trans1.bn1.running_var', 'trans1.conv1.weight', 'block2.layer.0.bn1.weight', 'block2.layer.0.bn1.bias', 'block2.layer.0.bn1.running_mean', 'block2.layer.0.bn1.running_var', 'block2.layer.0.conv1.weight', 'block2.layer.1.bn1.weight', 'block2.layer.1.bn1.bias', 'block2.layer.1.bn1.running_mean', 'block2.layer.1.bn1.running_var', 'block2.layer.1.conv1.weight', 'block2.layer.2.bn1.weight', 'block2.layer.2.bn1.bias', 'block2.layer.2.bn1.running_mean', 'block2.layer.2.bn1.running_var', 'block2.layer.2.conv1.weight', 'block2.layer.3.bn1.weight', 'block2.layer.3.bn1.bias', 'block2.layer.3.bn1.running_mean', 'block2.layer.3.bn1.running_var', 'block2.layer.3.conv1.weight', 'block2.layer.4.bn1.weight', 'block2.layer.4.bn1.bias', 'block2.layer.4.bn1.running_mean', 'block2.layer.4.bn1.running_var', 'block2.layer.4.conv1.weight', 'block2.layer.5.bn1.weight', 'block2.layer.5.bn1.bias', 'block2.layer.5.bn1.running_mean', 'block2.layer.5.bn1.running_var', 'block2.layer.5.conv1.weight', 'block2.layer.6.bn1.weight', 'block2.layer.6.bn1.bias', 'block2.layer.6.bn1.running_mean', 'block2.layer.6.bn1.running_var', 'block2.layer.6.conv1.weight', 'block2.layer.7.bn1.weight', 'block2.layer.7.bn1.bias', 'block2.layer.7.bn1.running_mean', 'block2.layer.7.bn1.running_var', 'block2.layer.7.conv1.weight', 'block2.layer.8.bn1.weight', 'block2.layer.8.bn1.bias', 'block2.layer.8.bn1.running_mean', 'block2.layer.8.bn1.running_var', 'block2.layer.8.conv1.weight', 'block2.layer.9.bn1.weight', 'block2.layer.9.bn1.bias', 'block2.layer.9.bn1.running_mean', 'block2.layer.9.bn1.running_var', 'block2.layer.9.conv1.weight', 'block2.layer.10.bn1.weight', 'block2.layer.10.bn1.bias', 'block2.layer.10.bn1.running_mean', 'block2.layer.10.bn1.running_var', 'block2.layer.10.conv1.weight', 'block2.layer.11.bn1.weight', 'block2.layer.11.bn1.bias', 'block2.layer.11.bn1.running_mean', 'block2.layer.11.bn1.running_var', 'block2.layer.11.conv1.weight', 'trans2.bn1.weight', 'trans2.bn1.bias', 'trans2.bn1.running_mean', 'trans2.bn1.running_var', 'trans2.conv1.weight', 'block3.layer.0.bn1.weight', 'block3.layer.0.bn1.bias', 'block3.layer.0.bn1.running_mean', 'block3.layer.0.bn1.running_var', 'block3.layer.0.conv1.weight', 'block3.layer.1.bn1.weight', 'block3.layer.1.bn1.bias', 'block3.layer.1.bn1.running_mean', 'block3.layer.1.bn1.running_var', 'block3.layer.1.conv1.weight', 'block3.layer.2.bn1.weight', 'block3.layer.2.bn1.bias', 'block3.layer.2.bn1.running_mean', 'block3.layer.2.bn1.running_var', 'block3.layer.2.conv1.weight', 'block3.layer.3.bn1.weight', 'block3.layer.3.bn1.bias', 'block3.layer.3.bn1.running_mean', 'block3.layer.3.bn1.running_var', 'block3.layer.3.conv1.weight', 'block3.layer.4.bn1.weight', 'block3.layer.4.bn1.bias', 'block3.layer.4.bn1.running_mean', 'block3.layer.4.bn1.running_var', 'block3.layer.4.conv1.weight', 'block3.layer.5.bn1.weight', 'block3.layer.5.bn1.bias', 'block3.layer.5.bn1.running_mean', 'block3.layer.5.bn1.running_var', 'block3.layer.5.conv1.weight', 'block3.layer.6.bn1.weight', 'block3.layer.6.bn1.bias', 'block3.layer.6.bn1.running_mean', 'block3.layer.6.bn1.running_var', 'block3.layer.6.conv1.weight', 'block3.layer.7.bn1.weight', 'block3.layer.7.bn1.bias', 'block3.layer.7.bn1.running_mean', 'block3.layer.7.bn1.running_var', 'block3.layer.7.conv1.weight', 'block3.layer.8.bn1.weight', 'block3.layer.8.bn1.bias', 'block3.layer.8.bn1.running_mean', 'block3.layer.8.bn1.running_var', 'block3.layer.8.conv1.weight', 'block3.layer.9.bn1.weight', 'block3.layer.9.bn1.bias', 'block3.layer.9.bn1.running_mean', 'block3.layer.9.bn1.running_var', 'block3.layer.9.conv1.weight', 'block3.layer.10.bn1.weight', 'block3.layer.10.bn1.bias', 'block3.layer.10.bn1.running_mean', 'block3.layer.10.bn1.running_var', 'block3.layer.10.conv1.weight', 'block3.layer.11.bn1.weight', 'block3.layer.11.bn1.bias', 'block3.layer.11.bn1.running_mean', 'block3.layer.11.bn1.running_var', 'block3.layer.11.conv1.weight', 'bn1.weight', 'bn1.bias', 'bn1.running_mean', 'bn1.running_var', 'fc.weight', 'fc.bias']


In [2]:
def main(arg_string):
    global args, best_prec1
    #setting up integrating the two models
    current_dir = '/datasets/home/05/205/jmattern/285_proj/'
#     current_dir = '/datasets/ee285s-public/'
#     model_path = os.path.join(current_dir,'models/DBPN_x2.pth')
#     upscale_factor = 8 # Can be 2, 4 or 8
    cuda = True # Set True if you're using GPU
    gpus=2

    seed = 123
    
    arg_list = arg_string.split()
    args = parser.parse_args(arg_list) #Adjusted for arguements
#     if args.tensorboard: configure("runs/%s"%(args.name))
    
    # Data loading code
    normalize = transforms.Normalize(mean=[x/255.0 for x in [125.3, 123.0, 113.9]],
                                     std=[x/255.0 for x in [63.0, 62.1, 66.7]])
    
    if args.augment:
        transform_train = transforms.Compose([
            transforms.RandomCrop(128, padding=4),
            transforms.RandomHorizontalFlip(),
#             transforms.Resize((64,64),Image.BICUBIC), #x2 scaling
            transforms.ToTensor(),
            normalize,
            ])
    else:
        transform_train = transforms.Compose([
            transforms.ToTensor(),
            transforms.
            normalize,
            ])
    transform_test = transforms.Compose([
#         transforms.Resize((64,64),Image.BICUBIC), #x2 scaling
        transforms.ToTensor(),
        normalize
        ])
    
    kwargs = {'num_workers': 1, 'pin_memory': True}
    train_loader = torch.utils.data.DataLoader(SRCIFAR100(os.path.join(current_dir,'X4CIFAR100'), transform_train), batch_size=args.batch_size, shuffle=True, **kwargs)
    val_loader = torch.utils.data.DataLoader(SRCIFAR100(os.path.join(current_dir,'X4CIFAR100_t'), transform_test), batch_size=args.batch_size, shuffle=True, **kwargs)
    train_file = '/train_x4.npy'
    test_file = '/test_x4.npy'
#     train_loader = torch.utils.data.DataLoader(
#         datasets.CIFAR100(os.path.join(current_dir), train=True, download=True,
#                          transform=transform_train),
#         batch_size=args.batch_size, shuffle=True, **kwargs)
#     val_loader = torch.utils.data.DataLoader(
#         datasets.CIFAR100(os.path.join(current_dir), train=False, transform=transform_test),
#         batch_size=args.batch_size, shuffle=True, **kwargs)

    # create model
    model = dn.DenseNet3(args.layers, 100, args.growth, reduction=args.reduce,
                         bottleneck=args.bottleneck, dropRate=args.droprate)
    
    model = model.cuda()


    # get the number of model parameters
    print('Number of model parameters: {}'.format(
        sum([p.data.nelement() for p in model.parameters()])))
    

    # optionally resume from a checkpoint
    if args.resume:
        if os.path.isfile(args.resume):
            print("=> loading checkpoint '{}'".format(args.resume))
            checkpoint = torch.load(args.resume)
            args.start_epoch = checkpoint['epoch']
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args.resume, checkpoint['epoch']))
        else:
            print("=> no checkpoint found at '{}'".format(args.resume))

    cudnn.benchmark = True
    
    # define loss function (criterion) and pptimizer
    criterion = nn.CrossEntropyLoss().cuda()
    optimizer = torch.optim.SGD(model.parameters(), args.lr,
                                momentum=args.momentum,
                                nesterov=True,
                                weight_decay=args.weight_decay)

    for epoch in range(args.start_epoch, args.epochs):
        adjust_learning_rate(optimizer, epoch)

        # train for one epoch
        prec2 = train(train_loader, model, criterion, optimizer, epoch)
        
        # evaluate on validation set
        prec1 = validate(val_loader, model, criterion, epoch)
        
        if (epoch == 0):
            np.save(current_dir + train_file,prec2)
            np.save(current_dir + test_file,prec1)
        else:
            train_acc = np.load(current_dir + train_file)
            test_acc = np.load(current_dir + test_file)
            t_val = prec2
            v_val = prec1
            np.save(current_dir + train_file,np.array(np.append(train_acc,t_val)))
            np.save(current_dir + test_file,np.array(np.append(test_acc,v_val)))
        
        # remember best prec@1 and save checkpoint
        is_best = prec1 > best_prec1
        best_prec1 = max(prec1, best_prec1)
#         print(model.state_dict())
        
        save_checkpoint({
            'epoch': epoch,
            'state_dict': model.state_dict(),
            'best_prec1': best_prec1,
        }, is_best)
    print('Best accuracy: ', best_prec1)
    
    train_acc = np.load(current_dir + train_file)
    test_acc = np.load(current_dir + test_file)
    return train_acc, test_acc

def train(train_loader, model, criterion, optimizer, epoch):
    """Train for one epoch on the training set"""
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    for i, (input, target) in enumerate(train_loader):
        target = target.cuda(async=True)
        input = input.cuda()
        input_var = torch.autograd.Variable(input)
        target_var = torch.autograd.Variable(target)

        
        # compute output
        output = model(input_var)
        loss = criterion(output, target_var)
        # measure accuracy and record loss
        prec1 = accuracy(output.data, target, topk=(1,))[0]
        losses.update(loss.data[0], input.size(0))
        top1.update(prec1[0], input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % args.print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                      epoch, i, len(train_loader), batch_time=batch_time,
                      loss=losses, top1=top1))
    return top1.avg
    # log to TensorBoard
#     if args.tensorboard:
#         log_value('train_loss', losses.avg, epoch)
#         log_value('train_acc', top1.avg, epoch)

def validate(val_loader, model, criterion, epoch):
    """Perform validation on the validation set"""
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, (input, target) in enumerate(val_loader):
        target = target.cuda(async=True)
        input = input.cuda()
        input_var = torch.autograd.Variable(input, volatile=True)
        target_var = torch.autograd.Variable(target, volatile=True)

        # compute output
        output = model(input_var)
        loss = criterion(output, target_var)

        # measure accuracy and record loss
        prec1 = accuracy(output.data, target, topk=(1,))[0]
        losses.update(loss.data[0], input.size(0))
        top1.update(prec1[0], input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % args.print_freq == 0:
            print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                      i, len(val_loader), batch_time=batch_time, loss=losses,
                      top1=top1))
        

    print(' * Prec@1 {top1.avg:.3f}'.format(top1=top1))
    # log to TensorBoard
#     if args.tensorboard:
#         log_value('val_loss', losses.avg, epoch)
#         log_value('val_acc', top1.avg, epoch)
    return top1.avg


def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    """Saves checkpoint to disk"""
    directory = "runs/%s/"%(args.name)
    if not os.path.exists(directory):
        os.makedirs(directory)
    filename = directory + filename
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'runs/%s/'%(args.name) + 'model_best.pth.tar')

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 after 150 and 225 epochs"""
    lr = args.lr * (0.1 ** (epoch // 150)) * (0.1 ** (epoch // 225))
    # log to TensorBoard
#     if args.tensorboard:
#         log_value('learning_rate', lr, epoch)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

In [3]:
new = False
if new:
    arpg_string = "--layers 20 --growth 12 -b 64 --no-bottleneck --reduce 1.0 --name DenseNet-20-12-x4"
else:
    arg_string = "--layers 20 --growth 12 -b 64 --no-bottleneck --reduce 1.0 --name DenseNet-20-12-x4 --resume runs/DenseNet-20-12-x4/checkpoint.pth.tar"
train_acc, test_acc = main(arg_string)
plt.figure()
plt.plot(train_acc)
plt.plot(test_acc)

Number of model parameters: 228004
=> loading checkpoint 'runs/DenseNet-20-12-x4/checkpoint.pth.tar'
=> loaded checkpoint 'runs/DenseNet-20-12-x4/checkpoint.pth.tar' (epoch 173)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:145: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:146: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch: [173][0/782]	Time 4.143 (4.143)	Loss 1.0350 (1.0350)	Prec@1 70.312 (70.312)
Epoch: [173][10/782]	Time 0.233 (0.570)	Loss 1.3908 (0.9557)	Prec@1 62.500 (74.006)
Epoch: [173][20/782]	Time 0.234 (0.420)	Loss 1.0153 (0.9214)	Prec@1 68.750 (74.107)
Epoch: [173][30/782]	Time 0.245 (0.365)	Loss 1.1835 (0.9426)	Prec@1 68.750 (73.690)
Epoch: [173][40/782]	Time 0.242 (0.337)	Loss 1.0521 (0.9368)	Prec@1 64.062 (72.790)
Epoch: [173][50/782]	Time 0.304 (0.321)	Loss 0.8766 (0.9432)	Prec@1 75.000 (72.641)
Epoch: [173][60/782]	Time 0.267 (0.310)	Loss 1.1315 (0.9415)	Prec@1 68.750 (72.925)
Epoch: [173][70/782]	Time 0.255 (0.302)	Loss 1.0050 (0.9315)	Prec@1 71.875 (73.283)
Epoch: [173][80/782]	Time 0.298 (0.296)	Loss 0.8708 (0.9321)	Prec@1 75.000 (73.052)
Epoch: [173][90/782]	Time 0.271 (0.293)	Loss 0.6884 (0.9255)	Prec@1 82.812 (73.249)
Epoch: [173][100/782]	Time 0.245 (0.289)	Loss 0.8012 (0.9307)	Prec@1 73.438 (72.989)
Epoch: [173][110/782]	Time 0.239 (0.285)	Loss 0.7411 (0.9287)	Prec@1 75.000 

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:183: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:184: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:192: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:193: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Test: [0/157]	Time 0.435 (0.435)	Loss 1.3748 (1.3748)	Prec@1 65.625 (65.625)
Test: [10/157]	Time 0.213 (0.228)	Loss 1.4087 (1.2928)	Prec@1 62.500 (63.778)
Test: [20/157]	Time 0.221 (0.224)	Loss 1.4644 (1.3689)	Prec@1 65.625 (62.351)
Test: [30/157]	Time 0.226 (0.226)	Loss 1.3917 (1.4013)	Prec@1 65.625 (62.147)
Test: [40/157]	Time 0.244 (0.227)	Loss 1.2531 (1.3898)	Prec@1 59.375 (62.195)
Test: [50/157]	Time 0.228 (0.227)	Loss 0.9713 (1.3888)	Prec@1 64.062 (61.612)
Test: [60/157]	Time 0.229 (0.227)	Loss 1.2781 (1.3661)	Prec@1 65.625 (62.193)
Test: [70/157]	Time 0.211 (0.226)	Loss 1.2229 (1.3691)	Prec@1 67.188 (62.214)
Test: [80/157]	Time 0.201 (0.223)	Loss 1.1887 (1.3627)	Prec@1 71.875 (62.133)
Test: [90/157]	Time 0.254 (0.224)	Loss 1.2052 (1.3609)	Prec@1 59.375 (62.174)
Test: [100/157]	Time 0.257 (0.225)	Loss 1.4711 (1.3667)	Prec@1 57.812 (62.113)
Test: [110/157]	Time 0.226 (0.228)	Loss 1.0227 (1.3676)	Prec@1 67.188 (62.148)
Test: [120/157]	Time 0.204 (0.227)	Loss 1.3895 (1.3593)	Prec@1 

...

Epoch: [200][620/782]	Time 0.238 (0.237)	Loss 0.8692 (0.9085)	Prec@1 71.875 (73.090)
Epoch: [200][630/782]	Time 0.235 (0.237)	Loss 1.0003 (0.9090)	Prec@1 71.875 (73.088)
Epoch: [200][640/782]	Time 0.240 (0.237)	Loss 1.1224 (0.9095)	Prec@1 59.375 (73.072)
Epoch: [200][650/782]	Time 0.235 (0.237)	Loss 0.8865 (0.9101)	Prec@1 78.125 (73.056)
Epoch: [200][660/782]	Time 0.238 (0.237)	Loss 0.9166 (0.9118)	Prec@1 68.750 (73.021)
Epoch: [200][670/782]	Time 0.240 (0.237)	Loss 1.0380 (0.9111)	Prec@1 75.000 (73.039)
Epoch: [200][680/782]	Time 0.238 (0.237)	Loss 0.9074 (0.9111)	Prec@1 75.000 (73.027)
Epoch: [200][690/782]	Time 0.232 (0.237)	Loss 0.8996 (0.9108)	Prec@1 71.875 (73.028)
Epoch: [200][700/782]	Time 0.237 (0.237)	Loss 1.0540 (0.9107)	Prec@1 70.312 (73.045)
Epoch: [200][710/782]	Time 0.236 (0.237)	Loss 0.9901 (0.9127)	Prec@1 70.312 (73.013)
Epoch: [200][720/782]	Time 0.235 (0.237)	Loss 1.0462 (0.9148)	Prec@1 75.000 (72.946)
Epoch: [200][730/782]	Time 0.237 (0.237)	Loss 1.0000 (0.9159)	Pre

Epoch: [201][650/782]	Time 0.237 (0.238)	Loss 0.9481 (0.9171)	Prec@1 73.438 (72.938)
Epoch: [201][660/782]	Time 0.237 (0.238)	Loss 0.8104 (0.9176)	Prec@1 73.438 (72.913)
Epoch: [201][670/782]	Time 0.237 (0.238)	Loss 0.9438 (0.9195)	Prec@1 75.000 (72.865)
Epoch: [201][680/782]	Time 0.234 (0.238)	Loss 0.8843 (0.9194)	Prec@1 70.312 (72.866)
Epoch: [201][690/782]	Time 0.236 (0.238)	Loss 1.2201 (0.9200)	Prec@1 67.188 (72.843)
Epoch: [201][700/782]	Time 0.237 (0.238)	Loss 0.8535 (0.9182)	Prec@1 76.562 (72.920)
Epoch: [201][710/782]	Time 0.232 (0.238)	Loss 0.9330 (0.9190)	Prec@1 76.562 (72.919)
Epoch: [201][720/782]	Time 0.248 (0.238)	Loss 0.7806 (0.9187)	Prec@1 70.312 (72.917)
Epoch: [201][730/782]	Time 0.238 (0.238)	Loss 0.9028 (0.9195)	Prec@1 71.875 (72.886)
Epoch: [201][740/782]	Time 0.232 (0.238)	Loss 0.9654 (0.9196)	Prec@1 73.438 (72.870)
Epoch: [201][750/782]	Time 0.242 (0.238)	Loss 0.9124 (0.9201)	Prec@1 70.312 (72.861)
Epoch: [201][760/782]	Time 0.238 (0.238)	Loss 1.1160 (0.9198)	Pre

Epoch: [202][680/782]	Time 0.242 (0.237)	Loss 1.2139 (0.9153)	Prec@1 67.188 (72.990)
Epoch: [202][690/782]	Time 0.237 (0.237)	Loss 1.1143 (0.9173)	Prec@1 62.500 (72.945)
Epoch: [202][700/782]	Time 0.238 (0.237)	Loss 0.9001 (0.9183)	Prec@1 75.000 (72.916)
Epoch: [202][710/782]	Time 0.235 (0.237)	Loss 1.0584 (0.9183)	Prec@1 67.188 (72.950)
Epoch: [202][720/782]	Time 0.240 (0.237)	Loss 0.9910 (0.9178)	Prec@1 67.188 (72.946)
Epoch: [202][730/782]	Time 0.239 (0.237)	Loss 0.6827 (0.9184)	Prec@1 68.750 (72.910)
Epoch: [202][740/782]	Time 0.236 (0.237)	Loss 0.9442 (0.9186)	Prec@1 71.875 (72.900)
Epoch: [202][750/782]	Time 0.246 (0.237)	Loss 0.7223 (0.9202)	Prec@1 76.562 (72.865)
Epoch: [202][760/782]	Time 0.237 (0.237)	Loss 0.9138 (0.9204)	Prec@1 73.438 (72.840)
Epoch: [202][770/782]	Time 0.234 (0.237)	Loss 0.7658 (0.9213)	Prec@1 75.000 (72.819)
Epoch: [202][780/782]	Time 0.240 (0.237)	Loss 0.8774 (0.9208)	Prec@1 70.312 (72.803)
Test: [0/157]	Time 0.313 (0.313)	Loss 1.3356 (1.3356)	Prec@1 60.9

Process Process-61:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 52, in _worker_loop
    r = index_queue.get()
  File "/opt/conda/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/opt/conda/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/opt/conda/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/opt/conda/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
current_dir = '/datasets/home/05/205/jmattern/285_proj/'
train_acc = np.load(current_dir + '/train_x4.npy')
test_acc = np.load(current_dir + '/test_x4.npy')
plt.figure()
plt.plot(100-train_acc)
plt.plot(100-test_acc)
plt.show()